# Change the configuration of a project

This notebook shows how to view and update the project configuration. By modifying the project settings, you can customize the behavior of Geti or adjust the training settings to achieve better results for your task.

## Initial setup

#### Connect to Geti

First, set up the connection to the server. This is done by instantiating a Geti object, with the hostname (or ip address) and authentication details for the server. As in notebook [001 create_project](001_create_project.ipynb), the server details are stored in the `.env` file and are loaded in the cell below. For details on how to create the `.env` file, please see the [readme](README.md).

In [ ]:
from geti_sdk import Geti
from geti_sdk.rest_clients import ProjectClient
from geti_sdk.utils import get_server_details_from_env

geti_server_configuration = get_server_details_from_env()

geti = Geti(server_config=geti_server_configuration)

project_client = ProjectClient(session=geti.session, workspace_id=geti.workspace_id)

#### Select a project
This example is based on the project `Cats and Dogs` created by the notebook [004](004_train_project.ipynb). Feel free to use a different project by changing the `PROJECT_NAME` variable.

In [ ]:
PROJECT_NAME = "Cats and Dogs"

project = project_client.get_project(PROJECT_NAME)

## Manage the configuration

#### Setup the ConfigurationClient

The class `ProjectConfigurationClient` offers convenient methods to retrieve and update the configuration of a project.

In [ ]:
from geti_sdk.rest_clients import ProjectConfigurationClient

configuration_client = ProjectConfigurationClient(
    workspace_id=geti.workspace_id, project=project, session=geti.session
)

#### Load the project configuration

Use the `get_configuration` method to load the configuration.

In [ ]:
project_config = configuration_client.get_configuration()

After loading the configuration, let's display it as JSON object, for better readability.

In [ ]:
import json

print(json.dumps(project_config.model_dump(), indent=4))

#### View the configuration for a specific task

The configuration object contains one section for each task in the project. To configure the settings for a specific task, use the `get_task_config` method.

In [ ]:
# In a task-chain project, you may change the index to select another task
task = project.get_trainable_tasks()[0]

task_config = project_config.get_task_config(task.id)

print(json.dumps(task_config.model_dump(), indent=4))

### Change the configuration values

The configuration can be updated with the method `set_configuration`, after updating the values of the configuration object. For example, you can change the minimum number of annotated images required for triggering auto-training as follows:

In [ ]:
task_config.auto_training.min_images_per_label = 24

configuration_client.set_configuration(project_config)

Now verify that the changes have taken effect by reloading the configuration.

In [ ]:
project_config = configuration_client.get_configuration()

new_value_after_reload = project_config.get_task_config(
    task.id
).auto_training.min_images_per_label

print(new_value_after_reload)

#### Disable auto-training

The configuration client also provides a handy method to enable/disable the auto-train functionality. This method is a quicker alternative to the process shown above.

In [ ]:
# Disable auto-train
configuration_client.set_project_auto_train(auto_train=False)

# Enable auto-train
configuration_client.set_project_auto_train(auto_train=True)